<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align=left>

# Identity: Entity and Groups
Vault client may have multiple accounts with various IdP that are enabled on the Vault server.  Vault clients can be mapped as **entities** and their corresponding accounts with authentication providers can be mapped as aliases.  In essence, each entity is made of zero or more aliases.
<img src=images/vault-identity-1.png width=600 align=left>

### Vault Setup

Prerequisities:
* Vault binary

Command to execute (in separate windows):
```
VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional
```

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault --version

### Apply License (recommended)

In [ ]:
export VAULT_TOKEN=root
curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_TOKEN" \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/v1/sys/license

In [ ]:
curl -s \
    --insecure \
    --header "X-Vault-Token: $VAULT_TOKEN" \
    $VAULT_ADDR/v1/sys/license  |jq ".data"
unset VAULT_TOKEN

In [ ]:
vault status

In [ ]:
unset VAULT_TOKEN
vault login root

**Note:** the Initialized and Sealed status above.

In [ ]:

vault auth enable userpass

vault policy write base - <<"EOF"
path "secret/data/training" {
   capabilities = [ "create", "read"]
}
EOF

vault policy write team-qa - <<"EOF"
path "secret/data/team/qa" {
   capabilities = [ "create", "read", "delete" ]
}
EOF

vault policy write test - <<"EOF"
path "secret/data/test" {
   capabilities = [ "create", "read", "update", "delete" ]
}
EOF

vault policy write team-eng - <<"EOF"
path "secret/data/team/eng" { 
   capabilities = [ "create", "read", "update", "delete" ]
} 
EOF


### Create Users
<img src="images/vault-entity-1.png">

Create two users (entity aliases) each having a different policy.

In [ ]:
vault write auth/userpass/users/bob password="training" policies="test"

In [ ]:
vault write auth/userpass/users/bsmith password="training" policies="team-qa"

In [ ]:
vault auth list
ACCESSOR=$(vault auth list -format=json | jq -r '.["userpass/"].accessor')
echo "\nuserpass/ accessor is $ACCESSOR"

### Create an Entity

In [ ]:
vault write -format=json identity/entity name="bob-smith" policies="base" \
   metadata=organization="ACME Inc." metadata=team="QA" | tee /tmp/bob-smith-entity.json

In [ ]:
cat /tmp/bob-smith-entity.json | jq -r ".data.id" > /tmp/bob-smith-entity_id.txt
cat /tmp/bob-smith-entity_id.txt

### Create Aliases for the Entity

In [ ]:
vault write identity/entity-alias name="bob" canonical_id=$(cat /tmp/bob-smith-entity_id.txt) mount_accessor=$ACCESSOR

In [ ]:
vault write identity/entity-alias name="bsmith" canonical_id=$(cat /tmp/bob-smith-entity_id.txt) mount_accessor=$ACCESSOR

In [ ]:
vault list identity/entity/id

**Check Entity's aliases**

In [ ]:
vault read identity/entity/id/$(cat /tmp/bob-smith-entity_id.txt) -format=json

**Note:** Alternatively, launch Vault UI http://localhost:8200/ui and go to **ACCESS > Entities** followed by **Aliases**

### Test the Entity
Understand how a token inherits the capabilities from entity's policy.

In [ ]:
vault login -method=userpass username=bob password=training

**Note:**  
* the token generated has `default` and `test` policies attached (`token_policies`)
* because `bob` is a member of `bob-smith` entity, he inherited `base` policy  (`identity_policies`)
* therefore `bob` is permitted to perform operations allowed by `base`, `test` and `default` policies

### Test Policies
`test` policy grants CRUD on `secret/data/test` path.

In [ ]:
vault kv put secret/test owner="bob"

In [ ]:
vault kv get secret/test

**Note:** Although `bob` and `bsmith` belong to the same person, allowed permissions may differ depending on the authentication account he uses to login.

In [ ]:
vault token capabilities secret/data/team/qa`

### Create an Internal Group
<img src="images/vault-identity-2.png" width=500>

In [ ]:
vault login root

In [ ]:
vault policy read team-eng

In [ ]:
vault write -format=json identity/group name="engineers2" policies="team-eng" \
   member_entity_ids=$(cat /tmp/bob-smith-entity_id.txt) \
   metadata=team="Engineering" metadata=region="North America" | tee /tmp/engineers.json

In [ ]:
cat /tmp/engineers.json | jq -r ".data.id" > /tmp/engineers-group_id.txt
cat /tmp/engineers-group_id.txt

**List existing groups by IDs**

In [ ]:
vault list identity/group/id

**List existing groups by Name**

In [ ]:
vault list identity/group/name

In [ ]:
vault read identity/group/name/engineers

**Note:** by default, Vault creates an **internal group**. When you create an internal group, you specify the group members.  You do not specify any group alias.  Group aliases are mapping between Vault and external IdP (e.g. LDAP, GitHub, etc.).
Therefore, you define group aliases only when you create **external groups**.  For internal groups, you have `member_entity_ids` and/or `member_group_ids` instead.

### Test the Group
Let's understand how a token inherits capabilities from its associating group.

In [ ]:
unset VAULT_TOKEN
vault login -method=userpass username="bsmith" password="training"

**Note:**  
* the token generated has `default` and `team-qa` policies attached (`token_policies`)
* because `bsmith` is a member of `bob-smith` entity, he inherited `base` policy  (`identity_policies`)
* because `bsmith` is a member of `team-qa` group, he inherited `team-qa` policy
* therefore `bsmith` is permitted to perform operations allowed by `base`, `default`, `team-eng`, and `team-qa` policies

In [ ]:
vault token capabilities secret/data/team/qa

**Note:** do the same for
```
1. secret/data/training_test
2. secret/data/team/qa
3. secret/data/team/eng
4. secret/data/test
```

### Create External Group and Group Alias
Test your Github token:
```
curl -H "Authorization: token <your_github_token>" https://api.github.com/user/teams
```


In [ ]:
vault login root > /dev/null

vault policy write education - <<"EOF"
path "secret/data/education" {
   capabilities = [ "create", "read", "update", "delete" , "list"]
}
EOF


### Github Auth

In [ ]:
vault login root

In [ ]:
vault auth enable github

In [ ]:
vault write auth/github/config organization=hashicorp

In [ ]:
vault auth list
GITHUB_ACCESSOR=$(vault auth list -format=json | jq -r '.["github/"].accessor')
echo "\ngithub accessor is $GITHUB_ACCESSOR"

### Create External Group

In [ ]:
vault write identity/group name="team-se" -format=json \
  type=external \
  policies="education" | tee /tmp/education.json
  

In [ ]:

EDU_GROUP_ID=$(cat /tmp/education.json | jq -r ".data.id" )
echo $EDU_GROUP_ID

### Create Group Alias

In [ ]:
vault login root > /dev/null
vault write identity/group-alias name="team-se" \
  mount_accessor=$GITHUB_ACCESSOR \
  canonical_id=$EDU_GROUP_ID

In [ ]:
curl -H "Authorization: token " https://api.github.com/user/teams

In [ ]:
vault login root
vault write auth/github/config organization=hashicorp

In [ ]:

vault login -method=github token=""

**Note:** I belong to `team-se` in GitHub and therefore to external group `team-se` which inherit `education` policy.  Therefore I can modify secret/data/education and nothing else.

In [ ]:
vault login root 
vault read identity/group/name/team-se -format=json

### License Vault

In [ ]:
vault token capabilities secret/data/training

In [ ]:
vault token capabilities secret/data/education

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">